In [5]:
import numpy as np
a = np.array([1,2,3])
b = np.array([4,5,6])
#c = np.sum(a*b)
c = np.dot(a,b)
c

32

In [12]:
a = np.array([[1,2,3],
             [4,5,6],
             [7,8,9]])
b = np.array([4,5,6])
c = np.dot(b,a)
print(c)
d = np.dot(a,b)
print(d)

[66 81 96]
[ 32  77 122]


In [22]:
a = np.array([[1,2,3]])
print(len(a))
print(a.shape)
b = np.array([[2],[2],[2]])
print(len(b))
print(b.shape)
c = np.dot(a,b)
print(c.ndim)
print(c.shape)
print(c)
c = np.dot(b,a)
print(c)

1
(1, 3)
3
(3, 1)
2
(1, 1)
[[12]]
[[2 4 6]
 [2 4 6]
 [2 4 6]]


In [26]:
a = np.array([[1,2,3],
              [4,5,6]])
print(a.shape)
b = np.array([[2],[2],[2]])
print(b.shape)
c = np.dot(a,b)
print(c)
c = np.dot(b,a)

(2, 3)
(3, 1)
[[12]
 [30]]


ValueError: shapes (3,1) and (2,3) not aligned: 1 (dim 1) != 2 (dim 0)

In [29]:
a = np.array([[1,2,3],
              [4,5,6]])
print(a.shape)
b = np.array([[1,2],
              [1,2],
              [1,2]])
print(b.shape)
c = np.dot(a,b)
print(c.shape)

(2, 3)
(3, 2)
(2, 2)


In [31]:
a = np.array([1,2,3,4,5])
print(a)
b = a
b[0] = 10
print(a)

[1 2 3 4 5]
[10  2  3  4  5]


In [33]:
a = np.array([1,2,3,4,5])
b = a.copy()
print(a)
print(b)
b[0]=10
print(a)
print(b)

[1 2 3 4 5]
[1 2 3 4 5]
[1 2 3 4 5]
[10  2  3  4  5]


In [39]:
a = np.array([1,2,3])
print(a.shape)
print(a.T)
print(a.T.shape)
b = np.reshape(a,(-1,1))
print(b.shape)

(3,)
[1 2 3]
(3,)
(3, 1)


In [42]:
a = np.array([[1,2,3],
              [1,2,3]])
print(a)
print(a.shape)
print(a.T)
print(a.T.shape)

[[1 2 3]
 [1 2 3]]
(2, 3)
[[1 1]
 [2 2]
 [3 3]]
(3, 2)


In [43]:
import numpy as np
import matplotlib.pyplot as plt

In [44]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [46]:
cancer = load_breast_cancer()
x = cancer.data
y = cancer.target

x_train_all, x_test, y_train_all, y_test = train_test_split(x,y,
                            stratify=y, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all,
                                    stratify=y_train_all, test_size=0.2, random_state=42)

In [48]:
print(x_train.shape)
print(y_train.shape)

(364, 30)
(364,)


In [51]:
a = x_train[:,1]
a = np.reshape(a,(-1,1))
print(a.shape)

(364, 1)


In [67]:
a = np.reshape(a,(a.shape[0],))
err = a-y_train
print(a.shape)
print(y_train.shape)
err = np.reshape(err,(err.shape[0],1))
print(err.shape)

(364,)
(364,)
(364, 1)


In [85]:
class SingleLayer:
    def __init__(self, learning_rate = 0.01,l1=0,l2=0):  ##
        self.w = None
        self.b = None
        self.losses = []
        self.w_history = []        
        self.lr = learning_rate    
        self.val_losses = []
        self.l1=l1
        self.l2=l2
        
    def forpass(self,x):
        #z = np.sum(x * self.w)+ self.b
        z = np.dot(x, self.w) + self.b
        return z
    
    def activation(self,z):
        z = np.clip(z,-100, None)
        a = 1 / (1 + np.exp(-z))
        return a
    
    def backprop(self, x, err):
        #w_grad = x * err
        #b_grad = 1 * err
        #mse
        m = len(x)
        w_grad = np.dot(x.T,err) /m
        b_grad = np.sum(err)/m
        return w_grad, b_grad
    
    def fit(self,x,y,epochs=100,x_val=None, y_val=None):
        y = y.reshape(-1,1)
        y_val = y_val.reshape(-1,1)
        m = len(x)
        self.w = np.ones((x.shape[1],1))
        
        self.b = 0
        self.w_history.append(self.w.copy())
        np.random.seed(42)
        for i in range(epochs):
            #loss = 0
            #indexes = np.random.permutation(np.arange(len(x)))
            #for i in indexes:
                #z = self.forpass(x[i])
                z = self.forpass(x)
                a = self.activation(z)
                err = a - y
                #w_grad,b_grad = self.backprop(x[i],err)
                w_grad,b_grad = self.backprop(x,err)
                
                
                w_grad += (self.l1 * np.sign(self.w) + self.l2 * self.w) / m
                self.w -= self.lr * w_grad           ##        
                self.b -= self.lr * b_grad
                
                self.w_history.append(self.w.copy())  
                a = np.clip(a, 1e-10, 1-1e-10)
                loss = np.sum(-(y*np.log(a) + (1-y)*np.log(1-a)))
                
                self.losses.append((loss + self.reg_loss())/m)  ##
                self.update_val_loss(x_val,y_val)
    def reg_loss(self):  
        return self.l1 * np.sum(np.abs(self.w)) + self.l2 / 2 * np.sum(self.w**2)  
    
    def update_val_loss(self,x_val,y_val):
        #if x_val is None:
        #    return
        #val_loss = 0
        #for i in range(len(x_val)):
        #    z = self.forpass(x_val[i])
            z = self.forpass(x_val)
            a = self.activation(z)
            a = np.clip(a,1e-10, 1-1e-10)
            val_loss = np.sum(-(y_val*np.log(a) + (1-y_val)*np.log(1-a)))
            self.val_losses.append((val_loss+ self.reg_loss())/len(y_val))
            
    def predict(self,x):
        #z = [self.forpass(x_i) for x_i in x]
        z = self.forpass(x)
        return z > 0
    
    def score(self,x,y):
        return np.mean(self.predict(x) == y.reshape(-1,1))

In [80]:
from sklearn.preprocessing import StandardScaler

In [81]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_val_scaled = scaler.transform(x_val)

In [87]:
single_layer = SingleLayer(l2=0.01)
single_layer.fit(x_train_scaled, y_train,
                x_val=x_val_scaled, y_val = y_val, epochs=1000)
single_layer.score(x_val_scaled,y_val)

0.9560439560439561